[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/044_cnn_transfer_learning/cnn_transfer_learning.ipynb)

# Transfer Learning en Redes Convolucionales

En posts anteriores hemos introducido la arquitectura de `red neuronal convolucional` y también hemos presentado varias arquitecturas famosas que han demostrado buenas prestaciones en multitud de tareas. Estas redes están formadas muchas capas convolucionales, algunas con más de 100 capas, lo cual significa que tienen muchos parámetros y entrenarlas desde cero puedes ser costoso. Sin embargo, existe una técnica que nos permite obtener buenos modelos con menores requisitos: el *transfer learning*. Ya hemos hablado anteriormente de esta técnica, en el contexto de modelos de lenguaje, pero la idea es la misma: utilizaremos el máximo número de capas de una red ya entrenada en otro dataset, y simplemente entrenaremos las nuevas capas que necesitemos para nuestra tarea concreta.

![](https://pennylane.ai/qml/_images/transfer_learning_general.png)

En este post vamos a ver cómo podemos utilizar una red neuronal pre-entrada en Imagenet, y adaptarla para una nueva tarea de clasificación con un pequeño dataset.

## El dataset

Nuestro objetivo será el de entrenar un clasificador de flores. Podemos descargar las imágenes de la siguiente url.

In [ ]:
# !pip install wget

In [ ]:
# import wget 

# wget.download('https://mymldatasets.s3.eu-de.cloud-object-storage.appdomain.cloud/flowers.zip')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import zipfile

# with zipfile.ZipFile('flowers.zip', 'r') as zip_ref:
#     zip_ref.extractall('.')
#descomprimimos nuestro data set
from zipfile import ZipFile
file_path = '/content/drive/MyDrive/Colab Notebooks/Vegetable Images Redes Convulsionales.zip'
with ZipFile(file_path, 'r') as zip:
  zip.extractall('.')

conjunto de datos de 15 tipos diferentes de vegetales comúnmente encontrados en todo el mundo. Los 15 tipos de vegetales incluidos en el conjunto de datos son frijoles, calabacín amargo, calabaza en forma de botella, berenjena, brócoli, col, pimiento, zanahoria, coliflor, pepino, papaya, patata, calabaza, rábano y tomate. El conjunto de datos consta de un total de 21,000 imágenes, donde cada clase contiene 1,400 imágenes de tamaño 224 x 224 en formato *.jpg.

El conjunto de datos se divide en tres partes para fines de entrenamiento, validación y prueba. El 70% del conjunto de datos se utiliza para el entrenamiento, el 15% se utiliza para la validación y el 15% restante se utiliza para la prueba. Este tipo de división de datos es común en el aprendizaje automático para garantizar que el modelo se entrene con una cantidad suficiente de datos, al tiempo que pueda generalizar bien a nuevos datos no vistos.

train (15000 images)
test (3000 images)
validation (3000 images)

Una vez extraído el dataset, podemos ver que tenemos 5 clases de flores diferentes, distribuidas en 5 carpetas diferentes. Cada carpeta contiene varios ejemplos de flores de la categoría en cuestión.

In [ ]:
import os 
#cargamos nuetro dataset una ves descomprimidad
#----------------------------
PATH = '/content/Vegetable Images/train'
#------------------------
classes = os.listdir(PATH)
classes

In [ ]:
imgs, labels = [], []

for i, lab in enumerate(classes):
  paths = os.listdir(f'{PATH}/{lab}')
  print(f'Categoría: {lab}. Imágenes: {len(paths)}')
  paths = [p for p in paths if p[-3:] == "jpg"]
  imgs += [f'{PATH}/{lab}/{img}' for img in paths]
  labels += [i]*len(paths)

  #visualizacion de nuestras clases que cada uno cuenta con 1000 imagenes que son de entrenamiento haciendo total de 15000 imagenes

In [ ]:
imgs[:5]
len(imgs)

Podemos visualizar algunas imágenes en el dataset.

In [ ]:
import random 
from skimage import io
import matplotlib.pyplot as plt

fig, axs = plt.subplots(4,8, figsize=(15,9))
for _ax in axs:
  for ax in _ax:
    ix = random.randint(0, len(imgs)-1)
    img = io.imread(imgs[ix])
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(classes[labels[ix]])
plt.show()

Vamos a crear también un subconjunto de test para poder comparar varios modelos.

In [ ]:
from sklearn.model_selection import train_test_split

train_imgs, test_imgs, train_labels, test_labels = train_test_split(imgs, labels, test_size=0.2, stratify=labels)

len(train_imgs), len(test_imgs)

Y por último creamos nuestros objetos `Dataset` y `DataLoader` para poder darle las imágenes a nuestros modelos.

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

class Dataset(torch.utils.data.Dataset):
  def __init__(self, X, y, trans, device):#devide gpu
    self.X = X
    self.y = y
    self.trans = trans
    self.device = device

  def __len__(self):
    return len(self.X)

  def __getitem__(self, ix):
    # cargar la imágen
    img = io.imread(self.X[ix])#lee la ruta de la  iamgen
    # aplicar transformaciones
    #print(f"imagen: {img}")
    if self.trans:#aplica transformaciones si es igual 
      img = self.trans(image=img)["image"]#verificar y leer las images
    return torch.from_numpy(img / 255.).float().permute(2,0,1), torch.tensor(self.y[ix])#devuelve tensores en torh img/255 normaliza la trandormar a  float

Nos aseguraremos que todas las imágenes del dataset tengan las mismas dimensiones: 224x224 píxeles.

In [ ]:
import albumentations as A #corta todas la images en un tamano especfico
#auqnue no fue necesario a causa que nuestro tamano de nuestras imagenes por defecto vienen de 224*224 se hizo un redimensionamiento
#para aseguar que nuestras imagenes esten todas del mismo tamano
#---------------------------------------------
trans = A.Compose([
    A.Resize(224,224)#realizamos para que las imagenes estn del mismo tammano es decir de 224*224
])

dataset = {
    'train': Dataset(train_imgs, train_labels, trans, device), 
    'test': Dataset(test_imgs, test_labels, trans, device)
}
#---------------------------------------------------

len(dataset['train']), len(dataset['test'])

In [ ]:
dataset['train'][2]#accede al tercer elemento de la lista (el índice 2) y devuelve su valor.

In [ ]:
fig, axs = plt.subplots(4,6, figsize=(14,8))
for _ax in axs:
  for ax in _ax:
    ix = random.randint(0, len(dataset['train'])-1)
    img, lab = dataset['train'][ix]
    ax.imshow(img.permute(1,2,0))
    ax.axis('off')
    ax.set_title(classes[lab])
plt.show()

In [ ]:
dataloader = {# se utiliza para cargar datos durante el entrenamiento y la evaluación de un modelo de aprendizaje automático.
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=64, shuffle=True, pin_memory=True), #bach_Size -> número de ejemplos de entrenamiento que se utilizarán en una iteración para ajustar los pesos del modelo.
    'test': torch.utils.data.DataLoader(dataset['test'], batch_size=256, shuffle=False)# La opción shuffle=True indica que los ejemplos de entrenamiento se barajarán aleatoriamente antes de dividirlos en lotes, lo que ayuda a garantizar que los lotes de entrenamiento no contengan ejemplos similares. 
 #pin_memory=True es una optimización que permite que los datos se transfieran más rápidamente a la memoria de la GPU durante el entrenamiento, si se está utilizando una GPU.
}

imgs, labels = next(iter(dataloader['train']))
imgs.shape

## El Modelo

Vamos a escoger la arquitectura `resnet`, de la que ya hablamos en el post anterior, para hacer nuestro clasificador. De este modelo usarmos todas las capas excepto la última, la cual sustituiremos por una nueva capa lineal para llevar a cabo la clasificación en 5 clases.

La principal ventaja de usar ResNet50 en lugar de ResNet18 es que es una red neuronal más profunda, con 50 capas en lugar de 18. Esto permite que ResNet50 aprenda representaciones más complejas y detalladas de las imágenes, lo que puede conducir a una mayor precisión en tareas de clasificación de imágenes. 

In [ ]:

#modelos de prueba utilizando resnet50 el objetivo de este modelo para utilizarlo fue que aprenda representaciones más complejas y 
#detalladas de las imágenes, lo que puede conducir a una mayor precisión en tareas de clasificación de imágenes.

import torchvision

resnet = torchvision.models.resnet50()
resnet



In [ ]:
class ModelCustom(torch.nn.Module):#El modelo se construye a partir de una red ResNet-18 pre-entrenada y se agrega una capa lineal en la parte superior para la clasificación final.
  def __init__(self, n_outputs=10, pretrained=False, freeze=False):
    super().__init__()
    # descargamos resnet
    resnet = torchvision.models.resnet18(pretrained=pretrained)
    # nos quedamos con todas las capas menos la última
    self.resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
    if freeze:
      for param in self.resnet.parameters():
        param.requires_grad=False
    # añadimos una nueva capa lineal para llevar a cabo la clasificación
    #-----------------------------------------------adecuamos los datos para el modelo con 15 salidad correspondientes a nuestros clases
    self.fc = torch.nn.Linear(512, 15)

  def forward(self, x):#resnet son los pesos,El método forward define la operación de propagación hacia adelante del modelo. Primero se pasa la entrada x a través de la red ResNet-18 y se aplana en un tensor de dos dimensiones. Luego, el tensor se pasa a través de una capa lineal para producir las salidas del modelo.
    x = self.resnet(x)
    x = x.view(x.shape[0], -1)#tranformacion en un numero de baches
    x = self.fc(x)
    return x

  def unfreeze(self):#El método unfreeze permite descongelar las capas de la red ResNet-18 para el entrenamiento de fine-tuning,
    for param in self.resnet.parameters():
        param.requires_grad=True#descongela la gradientes

In [ ]:
model_custom = ModelCustom()
outputs = model_custom(torch.randn(64, 3, 224, 224))
outputs.shape

In [ ]:
import os
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5, lr=1e-2):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")
  #--------------------------------------------------------------------------------------   
        # Guardar checkpoint en c
        checkpoint_path = f"checkpoint_epoch_{epoch}.pth"
        torch.save({
            'epoch': epoch, #el número de la época actual
            'model_state_dict': model.state_dict(),# diccionario que contiene los pesos del modelo actual
            'optimizer_state_dict': optimizer.state_dict(),#un diccionario que contiene el estado actual del optimizador.
            'loss': loss, #el valor de pérdida del último batch del entrenamiento.
        }, checkpoint_path)
  #----------------------------------------------------------------------------------

### Entrenando desde cero

En primer lugar vamos a entrenar nuestro modelo desde cero para ver qué métricas podemos obtener.

las activaciones son el resultado de aplicar una transformación no lineal a la suma ponderada de las entradas y los pesos de la capa.

In [ ]:
model_c = ModelCustom()
fit(model_c, dataloader, epochs=10)

se lleva a cabo durante 15 épocas y durante la última época, la pérdida de entrenamiento es 0.04503, la pérdida de validación es 0.08641, la precisión de entrenamiento es 0.98820 y la precisión de validación es 0.97482.

Como puedes ver es complicado conseguir buenas métricas ya que nuestro dataset es muy pequeño.

## Transfer Learning

es una técnica muy utilizada en el campo del aprendizaje profundo que permite reutilizar el conocimiento aprendido por una red neuronal en una tarea específica y aplicarlo a otra tarea relacionada.

Ahora vamos a entrenar el mismo caso pero, en este caso, utilizando los pesos pre-entrenados de `resnet`.



In [ ]:
#tomamos nuestros pesos ya entrenado del modelo resnet como tambien solo entrenamos al utltima capa
model_c = ModelCustom(pretrained=True, freeze=True)
fit(model_c, dataloader)

EN NUESTRO PRIMER ENTRENAMIENTO NOTAMOS QUE PRESENTAMOS CON 5 EPOCH UN RESULTADO DE CASI 98% TENEMOS UNA PERDIDAD DEL 12% Y UNA GANANCIAS DE VALIDACION DE 98%

Como puedes ver no sólo obtenemos un mejor modelo en menos *epochs* sino que además cada *epoch* tarda menos en completarse. Esto es debido a que, al no estar entrenando gran parte de la red, los requisitos computacionales se reducen considerablemente. Mejores modelos y entrenados más rápido.

## Fine Tuning
es una técnica que se utiliza en el aprendizaje profundo para mejorar el rendimiento de una red neuronal pre-entrenada para una tarea específica. El objetivo es ajustar los pesos de la red pre-entrenada en una tarea relacionada con la tarea que queremos resolver, permitiendo que la red aprenda a reconocer patrones más específicos en los datos de entrada.

Todavía podemos mejorar un poco más si, además de utilizar los pesos descargados de Imagenet en `resnet`, entrenamos también la red completa.

In [ ]:
#DE IGUAL FORMA TENEMOS EL FINE TUNING QUE NOS PERMITE UN MEJORA DE RENDIMIENTO
#DONDE INTRODUCIMOS LOS PESOS YA ENTRENADOS DE NUESTRO MODELO Y ADEMA ENTRENAMOS TODAS LAS CAPAS DEL MODELO
model_c = ModelCustom(pretrained=True, freeze=False)
fit(model_c, dataloader)

Es común entrenar primero el modelo sin entrenar la red pre-entrenada durante varias epochs y después seguir entrenando, pero permitiendo ahora la actualización de pesos también en la red pre-entrenada (usualmente con un *learning rate* más pequeño).

In [ ]:
model_o = ModelCustom(pretrained=True, freeze=True)
fit(model_o, dataloader,epochs=5)
model_o.unfreeze()
fit(model_o, dataloader, epochs=5, lr=1e-4)

Otra alternativa de *fine tuning* es la de entrenar el modelo con diferentes *learning rates*, uno para la red pre-entrenada y otro para las capas nuevas.

In [ ]:

#UTILIZAMOS Y PROBAMOS CON LERNIGN REIGHT DISTINTO AL PRINCIPO
optimizer = torch.optim.Adam([
    {'params': model_o.resnet.parameters(), 'lr': 1e-3},
    {'params': model_o.fc.parameters(), 'lr': 1e-2}
])

In [ ]:
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5, lr_resnet=1e-4, lr_fc=1e-3):
    model.to(device)
    optimizer = optimizer = torch.optim.Adam([{'params': model.resnet.parameters(), 'lr': lr_resnet},{'params': model.fc.parameters(), 'lr': lr_fc}])
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [ ]:
model_o = ModelCustom(pretrained=True, freeze=True)#pretraindes pesos del modelo, freeze=true que entrenamos la ultima cap
fit(model_o, dataloader, lr_resnet=1e-4, lr_fc=1e-3)
model_o.unfreeze()#ahora cambia a que entrenamoso todas las capas
fit(model_o, dataloader, lr_resnet=1e-4, lr_fc=1e-3)#entrenemos ahora con todas las capas

TENIENDO NUESTRO MODELO ENTRENADO CON LEARNING REIGTH DIFERENTE DONDE DA UN DICHOSO 99% DE PRECISION DE ENTRENAMIENTO Y UNA PERDIDA DE MENOS DE 1% PRESENTANDO UN EFICACIA DE ENTRENAMIENTO SOBRE LAS IMAGENES DE ESTE DATA SET

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualizacionEntrenamiento(model, dataloader):
    model.eval()
    count = 0
    fig, axs = plt.subplots(4, 5, figsize=(10, 8))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch
            X = X.to('cuda')
            y_pred = model(X)
            y_pred = torch.argmax(y_pred, axis=1)
            y = y.to('cuda')
            images = X.permute(0, 2, 3, 1).cpu().numpy()
            for i in range(images.shape[0]):
                if count >= 20:
                    break
                image = images[i]
                label = y[i].item()
                prediction = y_pred[i].item()
                row = count // 5
                col = count % 5
                axs[row, col].imshow(image)
                axs[row, col].set_title(f"img orig: {label},\n img predict: {prediction}")
                axs[row, col].axis('off')
                count += 1
            if count >= 20:
                break
    plt.show()

In [ ]:
visualizacionEntrenamiento(model_o, dataloader['test'])

## Resumen

En este post hemos visto como podemos llevar a cabo *transfer learning* con redes convolucionales. Aplicar esta técnica nos permitirá obtener mejores modelos con menos requisitos computacionales y con datasets reducidos. Podemos descargar una red pre-entrenada con otro dataset (idealmente, un dataset similar al nuestro) y aprovechar el máximo número de capas. Podemos *congelar* la red pre-entrenada, de manera que no se actualicen sus pesos durante el entrenamiento, y utilizarla solo como extractor de características que las nuevas capas (las cuales si entrenamos) pueden aprovechar. Aún así, hacer *fine tuning* (seguir entrenando la red pre-entrenada) puede dar como resultado un mejor modelo. El *transfer learning* es una técnica muy potente que siempre que podamos podemos aprovechar para reducir los requisitos computacionales de nuestros modelos.